In [2]:
!pip install -Uqqq


!pip -qqq install bitsandbytes accelerate

ERROR: You must give at least one requirement to install (see "pip help install")


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [3]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
prompt = """
Given a product comment in Persian, perform the following tasks:
1. Translate the comment into English.
2. Identify and list the aspects of the product mentioned in the comment.
3. Determine the sentiment (positive, negative, or neutral) associated with each identified aspect.

### Input:
Comment in Persian: "هدفون خوبیه قیمتش یکم بالاست ولی صداش کیفیت خوبی داره. در ضمن میکروفونش بعضی وقتا قطعی داره."

### Steps:
1. **Translation to English**:
   Persian Comment: "هدفون خوبیه قیمتش یکم بالاست ولی صداش کیفیت خوبی داره. در ضمن میکروفونش بعضی وقتا قطعی داره."
   English Translation:

2. **Aspect Extraction**:
   Identify the key aspects of the product mentioned in the English translation. Aspects are specific components, features, or attributes of the product.

   Extracted Aspects:

3. **Sentiment Analysis**:
   For each extracted aspect, determine the sentiment expressed in the comment. The sentiment can be positive, negative, or neutral.

   Aspect Sentiments:
   For each extracted aspect, determine the sentiment expressed in the comment. The sentiment can be positive, negative, or neutral.

## Example:
### Input:
  - Comment in Persian: "این گوشی واقعا عالیه. دوربین خیلی خوبه ولی باتری زود خالی میشه."

  - Output:

 ## Translation to English:
    Persian Comment: "این گوشی واقعا عالیه. دوربین خیلی خوبه ولی باتری زود خالی میشه."
    English Translation: "This phone is really great. The camera is very good, but the battery drains quickly."

 ## Aspect Extraction:
      Extracted Aspects:
        Phone
        Camera
        Battery

 ## Sentiment Analysis:
    Aspect Sentiments:
      Phone: Positive
      Camera: Positive
      Battery: Negative
"""

messages = [
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": """ Sure! I will do the following thing, send me your sentences to review them."""},
    {"role": "user", "content": "اصلا  ضد آفتاب خوبی نیست لکه میندازه رو پوست آدم و خیلی چربه. فقط قیمتش خوبه اونم با این کیفیت اصلا نمی ارزه"}
]

In [12]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 1000,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

 ### Translation to English:
  Persian Comment: "اصلا  ضد آفتاب خوبی نیست لکه میندازه رو پوست آدم و خیلی چربه. فقط قیمتش خوبه اونم با این کیفیت اصلا نمی ارزه"
  English Translation: "Naturally, the sun is not good because it has a huge body and a lot of fat. Only its price is good with this quality."

### Aspect Extraction:
  Extracted Aspects:
    Sun (implied to be negative due to the context)
    Body size (implied to be negative)
    Fatness (implied to be negative)
    Price (implied to be positive)

### Sentiment Analysis:
  Aspect Sentiments:
    Sun: Negative
    Body size: Negative
    Fatness: Negative
    Price: Positive
